# Revenue KPIs (USD)   

### Connection to the Orders Table  

In [1]:
# Libraries needed
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine

import json
from datetime import datetime, date, timedelta

# Libraries for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px


In [2]:
import creds_panoply

POSTGRES_ADDRESS = creds_panoply.address
POSTGRES_PORT = creds_panoply.port
POSTGRES_USERNAME = creds_panoply.username
POSTGRES_PASSWORD = creds_panoply.pw
POSTGRES_DBNAME = creds_panoply.dbname

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
    username=POSTGRES_USERNAME,
    password=POSTGRES_PASSWORD,
    ipaddress=POSTGRES_ADDRESS,
    port=POSTGRES_PORT,
    dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

### Sales - Today, Yesterday, last 7 days, MTD

In [3]:
# Importing selected fields
tbl_name = "cirkul_database_import.business_intelligence_orders"
sel_fields = "order_processed_at_date, order_revenue"
query = "SELECT " + sel_fields + " FROM " + tbl_name
df_orders = pd.read_sql_query(query, cnx)

# Field modification
df_orders['order_processed_at_date'] = df_orders['order_processed_at_date'].dt.date

# Today's sales
todays_orders = df_orders[df_orders['order_processed_at_date'] == date.today()]
today_rev =  todays_orders['order_revenue'].sum()
print("Today's Sales ($): ", '{:,.{prec}f}'.format(today_rev, prec =0))

# Yesterday's sales
y_days_orders = df_orders[df_orders['order_processed_at_date'] == date.today() - timedelta(days = 1)]
y_day_rev =  y_days_orders['order_revenue'].sum()
print("Yesterday's Sales ($): ", '{:,.{prec}f}'.format(y_day_rev, prec =0))

# Last 7 days
WTD_orders = df_orders[(df_orders['order_processed_at_date'] >= date.today() - timedelta(days = 7)) & \
    (df_orders['order_processed_at_date'] <= date.today())]
WTD_rev =  WTD_orders['order_revenue'].sum()
print("Last 7 days Sales ($): ", '{:,.{prec}f}'.format(WTD_rev, prec =0))

# MTD (Month till date)
first_day_of_month = datetime.today().replace(day=1)
first_day_of_month = first_day_of_month.date()
MTD_orders = df_orders[(df_orders['order_processed_at_date'] >= first_day_of_month) & \
    (df_orders['order_processed_at_date'] <= date.today())]
MTD_rev =  MTD_orders['order_revenue'].sum()
print("MTD Sales ($): ", '{:,.{prec}f}'.format(MTD_rev, prec =0))


Today's Sales ($):  153,290
Yesterday's Sales ($):  181,774
Last 7 days Sales ($):  1,476,823
MTD Sales ($):  3,665,192


### Daily Sales for last 60 days

In [4]:
# List of all orders in last 60 days
last60_orders = df_orders[(df_orders['order_processed_at_date'] >= date.today() - timedelta(days = 60)) & \
    (df_orders['order_processed_at_date'] <= date.today())]
# Grouping by day
last60_orders_daily = last60_orders.groupby('order_processed_at_date').sum().reset_index()
# Renaming columns
last60_orders_daily = last60_orders_daily.rename(columns={'order_processed_at_date':'Order Date', 'order_revenue':'Revenue'})

# Plot
fig = px.bar(data_frame=last60_orders_daily, x='Order Date', y='Revenue', title='Revenue - Last 60 days', \
    height=500, width=1000)
fig.update_xaxes(tickangle = -30)
fig.show()


## Monthly Sales

In [5]:
tbl_name = "cirkul_database_import.business_intelligence_orders"
sel_fields = "order_processed_at_date, order_revenue"
query = "SELECT " + sel_fields + " FROM " + tbl_name
df_orders = pd.read_sql_query(query, cnx)

df_orders['order_processed_at_date'] = df_orders['order_processed_at_date'].dt.strftime('%Y-%m')
monthly_revs = df_orders.groupby('order_processed_at_date').sum().reset_index()
monthly_revs = monthly_revs.rename(columns={'order_processed_at_date':'Month','order_revenue':'Revenue'})

# Plot
fig = px.bar(monthly_revs, x='Month', y='Revenue', title='Monthly Sales', \
    height=500, width=1000)
fig.show()


## Yearly Sales

In [6]:
tbl_name = "cirkul_database_import.business_intelligence_orders"
sel_fields = "order_processed_at_date, order_revenue"
query = "SELECT " + sel_fields + " FROM " + tbl_name
df_orders = pd.read_sql_query(query, cnx)

df_orders['order_processed_at_date'] = df_orders['order_processed_at_date'].dt.strftime('%Y')
yearly_revs = df_orders.groupby('order_processed_at_date').sum().reset_index()
yearly_revs = yearly_revs.rename(columns={'order_processed_at_date':'Year','order_revenue':'Revenue'})

# Plot
fig = px.bar(yearly_revs, x='Year', y='Revenue', title='Yearly Revenue', \
    height=500, width=1000)
fig.show()